In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
import pandas as pd
df=pd.read_csv('train_data.csv')

In [5]:
df.head()

,author,published_at,updated_at,like_count,comments
0,@patpatpat1336,2024-09-07T02:22:01Z,2024-09-07T02:22:01Z,0,I&#39;ve been playing clash of clans since it ...
1,@bewarethechameleon,2023-08-31T16:48:29Z,2023-08-31T16:48:29Z,0,"the first 10 things .... players hate video, h..."
2,@smiley6229,2023-06-05T10:28:46Z,2023-06-05T10:28:46Z,1,When it’s 2015 and you don’t have the builder ...
3,@mayankkumar1011,2023-05-29T18:48:43Z,2023-05-29T18:48:43Z,0,Played age of empires castle siege....having s...
4,@puuikien5866,2023-02-04T00:07:36Z,2023-02-04T00:07:36Z,0,HAHAHHAHAHAHBHAHABH THE SS ONE SO RELATAVLEOMG...


Initial Sentiment


In [7]:
from textblob import TextBlob

def get_sentiment(text):
    analysis = TextBlob(text)
    # Classify sentiment
    if analysis.sentiment.polarity > 0:
        return 1  # positive
    elif analysis.sentiment.polarity == 0:
        return 0  # neutral
    else:
        return -1  # negative

df['sentiment'] = df['comments'].apply(get_sentiment)


LowerCasing

In [8]:
df['comments'] = df['comments'].str.lower()


remove punctuation

In [9]:
df['comments'] = df['comments'].str.replace(r'[^\w\s]', '', regex=True)

URL,Mention,Markdown removal

In [10]:
import re
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)     # Remove mentions
    text = re.sub(r'#\w+', '', text)     # Remove hashtags
    text = re.sub(r'\d+', '', text)      # Remove numbers
    return text

df['comments'] = df['comments'].apply(clean_text)


Stopwords Removal

In [11]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['comments'] = df['comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


Negative word removal

In [12]:
from nltk.tokenize import word_tokenize

def handle_negation(text):
    tokens = word_tokenize(text)
    negation_flag = False
    handled_tokens = []

    for token in tokens:
        if token in ['not', 'no', 'never']:
            negation_flag = True
        elif negation_flag:
            handled_tokens.append('not_' + token)
            negation_flag = False
        else:
            handled_tokens.append(token)

    return ' '.join(handled_tokens)

df['comments'] = df['comments'].apply(handle_negation)

Tokenize

In [13]:
from nltk.tokenize import word_tokenize
df['comments'] = df['comments'].apply(word_tokenize)


Lemmatization

In [14]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
df['comments'] = df['comments'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['comments'] = df['comments'].apply(lambda x: ' '.join(x))


In [15]:
df.to_csv('final_train_clean_data.csv', index=False)


Vectorization

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['comments'])


Train_test_Split

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)
